In [1]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 50kB/s 
     |████████████████████████████████| 368kB 40.9MB/s 
     |████████████████████████████████| 3.2MB 48.8MB/s 
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1


In [2]:
import tensorflow as tf
import numpy as np

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


# Problem 1 (10 pts)

**CIFAR10 클래스가 다음과 같은 기능을 갖도록 코드를 작성하시오.**


-get_train_batch 함수로 batch_size 만큼의 학습데이터를 return한다.

-get_test_batch 함수로 Validation or Test 데이터를 return한다.
*   is_val=True 이면 Validation 데이터를, False이면 Test데이터를 return한다.
*   split_num개만큼 데이터를 split하여 list 형태로 return한다. ex) Data : numpy array [1,2,3,4,5,6], split_num : 2, -> return : [[1,2,3], [4,5,6]]

-epochs와 iterations 변수를 통해 현재 epoch과 iteration 정보를 얻을 수 있다.
*   ex) cifar10 = CIFAR10(), print(cifar10.epochs) -> 20

-reset 함수를 통해 epoch과 iteration 정보를 초기화 시킨다. (새로운 모델에 처음부터 다시 학습을 시작할 때 사용)

In [0]:
class CIFAR10:
  def __init__(self):
    self.X_train = np.load("/gdrive/My Drive/숙제4/CIFAR10_datasets/CIFAR10_X_train.npy")
    self.Y_train = np.load("/gdrive/My Drive/숙제4/CIFAR10_datasets/CIFAR10_y_train.npy")
    self.X_test = np.load("/gdrive/My Drive/숙제4/CIFAR10_datasets/CIFAR10_X_test.npy")
    self.Y_test = np.load("/gdrive/My Drive/숙제4/CIFAR10_datasets/CIFAR10_y_test.npy")

    self.X_val = self.X_train[45000:]
    self.Y_val = self.Y_train[45000:]
    self.X_train = self.X_train[:45000]
    self.Y_train = self.Y_train[:45000]

    self.idx_flag = 0
    self.epochs = 0
    self.iterations = 0

  def get_train_batch(self, batch_size=64):
    X_batch, Y_batch = None, None,
    ####################### Put your code here ###############
    """Returns training batch from dataset"""
    start_index = self.idx_flag
    end_index = start_index + batch_size

    data_count = 45000

    if start_index > data_count and end_index > data_count:
        start_index %= data_count
        end_index %= data_count
        self.epochs+=1
        X_batch = self.X_train[start_index:end_index]
        Y_batch = self.Y_train[start_index:end_index]
    elif end_index > data_count:
        end_index %= data_count
        self.epochs+=1
        X_batch = np.concatenate((self.X_train[start_index:], self.X_train[0:end_index]))
        Y_batch = np.concatenate((self.Y_train[start_index:], self.Y_train[0:end_index]))
    else:
        X_batch = self.X_train[start_index:end_index]
        Y_batch = self.Y_train[start_index:end_index]
    self.idx_flag=end_index
    self.iterations+=1;

    ##########################################################
    return X_batch, Y_batch

  def get_test_batch(self, is_val=True, split_num=100):
    split_X, split_Y = None, None
    ####################### Put your code here ###############
    #. ex) Data : numpy array [1,2,3,4,5,6], split_num : 2, -> return : [[1,2,3], [4,5,6]]

    if is_val == True:
      split_X = np.asarray(np.split(self.X_val,split_num)).tolist()
      split_Y = np.asarray(np.split(self.Y_val,split_num)).tolist()
    else:
      split_X = np.asarray(np.split(self.X_test,split_num)).tolist()
      split_Y = np.asarray(np.split(self.Y_test,split_num)).tolist()

    ##########################################################
    return split_X, split_Y
  
  def reset(self):
    ####################### Put your code here ###############
    self.epochs = 0
    self.iterations = 0
    self.idx_flag = 0
    ##########################################################

# Problem 2 (20 pts)
**tensorflow 라이브러리를 이용하여 다음 함수들을 작성하시오.**

In [0]:
def conv_layer(input, output_channel, filter_size=2, stride=1, padding='same', use_bias=True):
  output = None
  ####################### Put your code here ###############
  output=tf.keras.layers.Conv2D(output_channel,filter_size,strides=stride,padding=padding,use_bias=use_bias)(input)
  ##########################################################
  return output

def fc_layer(input, output_size, use_bias=True):
  output = None
  ####################### Put your code here ###############
  #units	Positive integer, dimensionality of the output space.
  output=tf.layers.dense(input,output_size,use_bias=True);
  ##########################################################
  return output

def squeeze_excitation_block(input, ratio=16):
  output = None
  ####################### Put your code here ###############
  #units = output_size 부재 -> output.shape[0] 으로 임시로 쓰고 있고 있음
  output=global_average_pooling(input)
  output=fc_layer(output,(int)((int)(input.shape[3])/ratio)); # output_size가 들어가야 함
  output=relu(output);
  output=fc_layer(output,(int)(input.shape[3]))
  output=sigmoid(output)
  output=tf.reshape(output,[-1,1,1,input.shape[3]])
  output=input*output
  ##########################################################
  return output

def relu(input):
  output = None
  ####################### Put your code here ###############
  output=tf.nn.relu(input)
  ##########################################################
  return output

def sigmoid(input):
  output = None
  ####################### Put your code here ###############
  output=tf.nn.sigmoid(input)
  ##########################################################
  return output

def global_average_pooling(input):
  output = None
  ####################### Put your code here ###############
  output=tf.keras.layers.GlobalAveragePooling2D()(input)
  ##########################################################
  return output

def max_pooling(input, filter_size=2, stride=2, padding="same"):
  output = None
  ####################### Put your code here ###############
  #output=tf.nn.max_pool(input,filter_size,stride,padding)
  output = tf.layers.max_pooling2d(input, filter_size, stride,padding)
  ##########################################################
  return output
  
def flatten_block(input):
  output = None
  ####################### Put your code here ###############
  output=tf.layers.flatten(input)
  ##########################################################
  return output

def batch_norm(input, is_training):
  output = None
  ####################### Put your code here ###############
  output = tf.layers.batch_normalization(input,is_training)
  ##########################################################
  return output

# Problem 3 (20 pts)

Problem 2에서 작성한 함수들을 이용하여 다음 조건을 만족하는 CNN 모델을 작성하시오.
*   Validation accuracy가 70% 이상인 네트워크.
*   use_SE=True이면 Squeeze and Excitation block이 추가된 모델을, False이면 추가되지 않은 모델을 return.
*   두개의 모델은 SE block의 유무를 제외한 모든 세팅이 같아야함. (하이퍼 파라미터, SE block 이외의 layer 구조 등)

In [0]:
def cnn_model(X, Y, is_training, use_SE=True):
  loss, accuracy, backprop = None, None, None
  ####################### Put your code here ###############
  #처음 레이어 [3, 3, 1, 32]
  # Learning rate
  learning_rate=0.0005
  if(use_SE==True):
    model=conv_layer(X,32,filter_size=3)
    model=squeeze_excitation_block(model)
    model=relu(model)
    model=conv_layer(model,64,filter_size=3)
    model=squeeze_excitation_block(model)
    model=relu(model)
    model=max_pooling(model)
    model=conv_layer(model,128,filter_size=3)
    model=squeeze_excitation_block(model)
    model=relu(model)
    model=max_pooling(model)
    model=conv_layer(model,128,filter_size=2)
    model=squeeze_excitation_block(model)
  else:
    model=conv_layer(X,32,filter_size=3)
    model=relu(model)
    model=conv_layer(model,64,filter_size=3)
    model=relu(model)
    model=max_pooling(model)
    model=conv_layer(model,128,filter_size=3)
    model=relu(model)
    model=max_pooling(model)
    model=conv_layer(model,128,filter_size=2)
  
  model=relu(model)
  model=max_pooling(model)
  model=flatten_block(model)
  model=fc_layer(model,1000)
  model=relu(model)
  model=fc_layer(model,500)
  model=relu(model)
  model=fc_layer(model,100)
  model=fc_layer(model,10)
  Y_one_hot = tf.squeeze(tf.one_hot(Y,10))
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(Y,10),logits=model))
  backprop = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

  prediction = tf.equal(tf.argmax(model,axis=1),tf.argmax(tf.squeeze(tf.one_hot(Y,10)),axis=1))
  accuracy = tf.reduce_mean(tf.cast(prediction,tf.float32))
  ##########################################################
  return loss, accuracy, backprop

# Problem 4 (20 pts)

Problem 3에서 작성된 모델을 학습하고 테스트 하는 함수를 작성하여라.

In [0]:
def train_model(sess, cifar10, X, Y, is_training, loss, accuracy, backprop, epochs=10, batch_size=32, check_iter=100, split_num=100):
  train_loss_list, validation_loss_list, train_acc_list, validation_acc_list = [], [], [], []
  while cifar10.epochs < epochs:
    l, acc = None, None
    ####################### Put your code here ###############
    X_batch, Y_batch = cifar10.get_train_batch(batch_size=batch_size)      
    l = loss.eval(feed_dict={X:X_batch, Y:Y_batch})
    acc = accuracy.eval(feed_dict={X:X_batch, Y:Y_batch})
    sess.run(backprop,feed_dict={X:X_batch, Y:Y_batch})
    ##########################################################
    if cifar10.iterations%check_iter == 0:
      val_loss, val_acc = test_model(sess, cifar10, X, Y, is_training, loss, accuracy, split_num, is_val=True)
      print('Epochs : %d, Iterations : %d, Train_loss : %f, Train_accuracy : %.3f, Val_loss : %f, Val_accuracy : %.3f'%\
            (cifar10.epochs, cifar10.iterations, l, acc, val_loss, val_acc))
      train_loss_list.append(l)
      validation_loss_list.append(val_loss)
      train_acc_list.append(acc)
      validation_acc_list.append(val_acc)
      
  return train_loss_list, validation_loss_list, train_acc_list, validation_acc_list

def test_model(sess, cifar10, X, Y, is_training, loss, accuracy, split_num, is_val):
  X_list, Y_list = cifar10.get_test_batch(is_val=is_val, split_num=split_num)
  accs, losses = None, None
  ####################### Put your code here ###############
  accs=0
  losses=0
  ##########################################################
  for X_batch, Y_batch in zip(X_list, Y_list):
    ####################### Put your code here ###############
    losses = losses + loss.eval(feed_dict={X:X_batch, Y:Y_batch})
    accs = accs + accuracy.eval(feed_dict={X:X_batch, Y:Y_batch})
    ##########################################################
  ####################### Put your code here ###############
  losses = losses / split_num
  accs = accs / split_num
  ##########################################################
  return losses, accs

# Problem 5 (20 pts)

SE block이 없는 CNN 모델과, SE block이 추가된 CNN 모델을 각각 학습하시오. (단, 아래의 조건을 만족하여야 함)
*   둘중 적어도 하나의 모델은 Validation accuracy 70% 이상. (도달하지 못한다면 모델을 Problem 3의 모델을 수정하시오)
*   두개의 모델은 SE block의 유무를 제외한 모든 세팅이 같아야함. (하이퍼 파라미터, SE block 이외의 layer 구조 등)

# CNN training and test without Squeeze and Excitation block

In [19]:
cifar10 = CIFAR10()

X = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3])
Y = tf.placeholder(dtype=tf.int64, shape=[None])
is_training = tf.placeholder(tf.bool)

loss, accuracy, backprop = cnn_model(X, Y, is_training, use_SE=False)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  result_without_SE = train_model(sess, cifar10, X, Y, is_training, loss, accuracy, backprop, epochs=100, check_iter=300, batch_size=128)#epochs=100
  test_loss, test_acc = test_model(sess, cifar10, X, Y, is_training, loss, accuracy, split_num=100, is_val=False)
  print("Test_loss : %f, Test_accuracy : %.3f"%(test_loss, test_acc))

sess.close()

Epochs : 0, Iterations : 300, Train_loss : 1.767785, Train_accuracy : 0.375, Val_loss : 1.624755, Val_accuracy : 0.412
Epochs : 1, Iterations : 600, Train_loss : 1.436143, Train_accuracy : 0.531, Val_loss : 1.478779, Val_accuracy : 0.489
Epochs : 2, Iterations : 900, Train_loss : 0.980033, Train_accuracy : 0.656, Val_loss : 1.109155, Val_accuracy : 0.622
Epochs : 3, Iterations : 1200, Train_loss : 0.722359, Train_accuracy : 0.711, Val_loss : 0.888030, Val_accuracy : 0.707
Epochs : 4, Iterations : 1500, Train_loss : 0.537882, Train_accuracy : 0.820, Val_loss : 0.969638, Val_accuracy : 0.691
Epochs : 5, Iterations : 1800, Train_loss : 0.411475, Train_accuracy : 0.820, Val_loss : 1.134713, Val_accuracy : 0.704
Epochs : 5, Iterations : 2100, Train_loss : 0.350997, Train_accuracy : 0.875, Val_loss : 1.309448, Val_accuracy : 0.687
Epochs : 6, Iterations : 2400, Train_loss : 0.353956, Train_accuracy : 0.898, Val_loss : 1.264533, Val_accuracy : 0.713
Epochs : 7, Iterations : 2700, Train_loss :

# CNN training and test with Squeeze and Excitation block

In [21]:
cifar10.reset()
tf.reset_default_graph()

X = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3])
Y = tf.placeholder(dtype=tf.int64, shape=[None])
is_training = tf.placeholder(tf.bool)

loss, accuracy, backprop = cnn_model(X, Y, is_training, use_SE=True)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  result_with_SE = train_model(sess, cifar10, X, Y, is_training, loss, accuracy, backprop, epochs=100, check_iter=300, batch_size=128) #epoch 100
  test_loss, test_acc = test_model(sess, cifar10, X, Y, is_training, loss, accuracy, split_num=100, is_val=False)
  print("Test_loss : %f, Test_accuracy : %.3f"%(test_loss, test_acc))

sess.close()

Instructions for updating:
Use tf.cast instead.
Epochs : 0, Iterations : 300, Train_loss : 1.877187, Train_accuracy : 0.375, Val_loss : 1.527053, Val_accuracy : 0.463
Epochs : 1, Iterations : 600, Train_loss : 1.111059, Train_accuracy : 0.586, Val_loss : 1.118556, Val_accuracy : 0.614
Epochs : 2, Iterations : 900, Train_loss : 0.894166, Train_accuracy : 0.656, Val_loss : 1.044024, Val_accuracy : 0.642
Epochs : 3, Iterations : 1200, Train_loss : 0.791275, Train_accuracy : 0.766, Val_loss : 0.884155, Val_accuracy : 0.701
Epochs : 4, Iterations : 1500, Train_loss : 0.592210, Train_accuracy : 0.797, Val_loss : 0.890688, Val_accuracy : 0.707
Epochs : 5, Iterations : 1800, Train_loss : 0.284353, Train_accuracy : 0.922, Val_loss : 0.992578, Val_accuracy : 0.720
Epochs : 5, Iterations : 2100, Train_loss : 0.319196, Train_accuracy : 0.906, Val_loss : 1.145264, Val_accuracy : 0.707
Epochs : 6, Iterations : 2400, Train_loss : 0.262737, Train_accuracy : 0.898, Val_loss : 1.507309, Val_accuracy : 0

# Problem 6 (10 pts)

matplotlib와 result_with_SE, result_without_SE 를 이용하여 다음 learning curve들을 plot 하시오. (여러개의 cell을 사용하여도 무방)

*   SE 사용 네트워크와 SE 미사용 네트워크의 training loss
*   SE 사용 네트워크와 SE 미사용 네트워크의 validation loss
*   SE 사용 네트워크와 SE 미사용 네트워크의 validation accuracy
*   SE 사용 네트워크의 training loss와 validation loss
*   SE 사용 네트워크의 training accuracy와 validation accuracy
*   SE 미사용 네트워크의 training loss와 validation loss
*   SE 미사용 네트워크의 training accuracy와 validation accuracy



In [26]:
import matplotlib.pyplot as plt

plt.figure(figsize=(200,100))

#SE 사용 네트워크와 SE 미사용 네트워크의 training loss
plt.subplot(331)
plt.title('SE with & SE without training loss',fontsize=20)
plt.xlabel('Iteration')
plt.ylabel('training loss')
#SE 사용 네트워크
plt.plot(range(len(result_with_SE[0])), result_with_SE[0],label='USE SE',color='skyblue')
#SE 미사용 네트워크
plt.plot(range(len(result_without_SE[0])), result_without_SE[0],label='NOT USE',color='pink')
plt.legend()

#SE 사용 네트워크와 SE 미사용 네트워크의 validation loss
plt.subplot(332)
plt.title('SE with & SE without validation loss',fontsize=20)
plt.xlabel('Iteration')
plt.ylabel('validation loss')
#SE 사용 네트워크
plt.plot(range(len(result_with_SE[1])), result_with_SE[1],label='USE SE',color='skyblue')
#SE 미사용 네트워크
plt.plot(range(len(result_without_SE[1])), result_without_SE[1],label='NOT USE',color='pink')
plt.legend()

#SE 사용 네트워크와 SE 미사용 네트워크의 validation loss
plt.subplot(333)
plt.title('SE with & SE without validation accuracy',fontsize=20)
plt.xlabel('Iteration')
plt.ylabel('validation accuracy')
#SE 사용 네트워크
plt.plot(range(len(result_with_SE[3])), result_with_SE[3],label='USE SE',color='skyblue')
#SE 미사용 네트워크
plt.plot(range(len(result_without_SE[3])), result_without_SE[3],label='NOT USE',color='hotpink')
plt.legend()

# SE 사용 네트워크의 training loss와 validation loss
plt.subplot(334)
plt.title('SE With training loss validation loss',fontsize=20)
plt.xlabel('Iteration')
plt.ylabel('Loss')
#SE 사용 네트워크
plt.plot(range(len(result_with_SE[0])), result_with_SE[0],label='training loss',color='skyblue')
#SE 미사용 네트워크
plt.plot(range(len(result_with_SE[1])), result_with_SE[1],label='validation loss',color='pink')
plt.legend()

# SE 사용 네트워크의 training accuracy와 validation accuracy
plt.subplot(335)
plt.title('SE With training accuracy와 validation accuracy',fontsize=20)
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
#SE 사용 네트워크
plt.plot(range(len(result_with_SE[2])), result_with_SE[2],label='training accuracy',color='skyblue')
#SE 미사용 네트워크
plt.plot(range(len(result_with_SE[3])), result_with_SE[3],label='validation accuracy',color='pink')
plt.legend()

#SE 미사용 네트워크의 training loss와 validation loss
plt.subplot(337)
plt.title('SE Without training loss와 validation loss',fontsize=20)
plt.xlabel('Iteration')
plt.ylabel('Loss')
#SE 사용 네트워크
plt.plot(range(len(result_without_SE[0])), result_without_SE[0],label='training loss',color='skyblue')
#SE 미사용 네트워크
plt.plot(range(len(result_without_SE[1])), result_without_SE[1],label='validation loss',color='pink')
plt.legend()

#SE 미사용 네트워크의 training accuracy와 validation accuracy
plt.subplot(338)
plt.title('SE Without training accuracy와 validation accuracy',fontsize=20)
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
#SE 사용 네트워크
plt.plot(range(len(result_without_SE[2])), result_without_SE[2],label='training accuracy',color='skyblue')
#SE 미사용 네트워크
plt.plot(range(len(result_without_SE[3])), result_without_SE[3],label='validation accuracy',color='pink')
plt.legend()

/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 48120 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 49324 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:214: RuntimeWarning: Glyph 50857 missing from current font.
  font.set_text(s, 0.0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:183: RuntimeWarning: Glyph 48120 missing from current font.
  font.set_text(s, 0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:183: RuntimeWarning: Glyph 49324 missing from current font.
  font.set_text(s, 0, flags=flags)
/usr/local/lib/python3.6/dist-packages/matplotlib/backends/backend_agg.py:183: RuntimeWarning: Glyph 50857 missing from current font.
  font.set_text